# Imaging Exoplanets Tutorial

We will use the `pyklip` package to subtract off the glare of the star and measure the position and brightness of the exoplanet

In [ ]:
!pip install pyklip

# Setup

## Download data from an epoch

In the example below, you will see code blocks to download the data from 2009 and 2021. Start with one, before the other. 

In [ ]:
!mkdir Project_Materials_2009
!wget -O Project_Materials_2009/center_im.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2009/center_im.fits
!wget -O Project_Materials_2009/median_unsat.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2009/median_unsat.fits
!wget -O Project_Materials_2009/rotnorth.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2009/rotnorth.fits

# !mkdir Project_Materials_2021
# !wget -O Project_Materials_2021/center_im.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2021/center_im_crop.fits
# !wget -O Project_Materials_2021/median_unsat.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2021/median_unsat.fits
# !wget -O Project_Materials_2021/rotnorth.fits https://www.astro.uvic.ca/~wthompson/posts/direct-image-data/Project_Materials_2021/rotnorth.fits

## Import Packages

In [ ]:
import numpy as np
import astropy.io.fits as fits
import pyklip.parallelized
import pyklip.instruments.Instrument as Instrument
import pyklip.fakes

import matplotlib.pylab as plt
%matplotlib inline

# Read in the necessary data

We will read in three different files:

 1. A time series of the science data. This is a 3-D image cube consisting of images of the system taken over time. The sky rotates over this period of time due to angular differential imaging. If you look really carefully, you can actually see a couple of planets! 
 2. An image of the star not blocked by the coronagraph, so we can measure the flux of the star. This is needed to measure flux ratio between the planet and the star.
 3. A 1-D array of parallactic angles. These angles specify the rotation of the sky for each frame in the 3-D image cube due to the Earth's rotation. These are needed to derotate the images so we can stack the signal of the planet together on the same pixel. 

In [ ]:
# load in the science frames for imaging the planet
with fits.open("Project_Materials_2009/center_im.fits") as hdulist:
    img_cube = hdulist[0].data # Series of images of the star system taken in time 
    exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']
    
# the location of the star behind the coronagraph based on the documentaion
star_centx = (img_cube.shape[2]-1)/2
star_centy = (img_cube.shape[1]-1)/2

# load in the calibration frame to calibrate the brightness of any sources with respect to the star
with fits.open("Project_Materials_2009/median_unsat.fits") as hdulist:
    calib_frame = hdulist[0].data # image of the unsaturated star for photometric calibration
    calib_exptime = hdulist[0].header['ITIME'] * hdulist[0].header['COADDS']

# the location of the star in the calibration frame
calib_centx = (calib_frame.shape[1]-1)/2
calib_centy = (calib_frame.shape[0]-1)/2

# the parallactic angles corresponding to each frame for angular differential imaging
with fits.open("Project_Materials_2009/rotnorth.fits") as hdulist:
    rot_angles = hdulist[0].data # rotation angle of the night sky for each science frame
    
plt.figure()
plt.imshow(img_cube[0], cmap="inferno")
plt.xlim([star_centx-150, star_centx+150])
plt.ylim([star_centy-150, star_centy+150])
plt.title("Science Frame")

plt.figure()
plt.imshow(calib_frame, cmap="inferno")
plt.xlim([calib_centx-150, calib_centx+150])
plt.ylim([calib_centy-150, calib_centy+150])
plt.title("Calibration Frame")

# Load in the data into `pyklip`

We need to specify the location of the star in each frame, and then pass the data into the `pyklip` framework, which standardizes data from many high-contrast imaging instruments

In [ ]:
centers = np.array([[star_centx, star_centy] for _ in range(img_cube.shape[0])])

dataset = Instrument.GenericData(img_cube, centers, parangs=-rot_angles)
dataset.OWA = 250

# Subtract off the stellar PSF

This parallelized code runs "KLIP" (basically PCA) to remove the glare of the star. Note that is might take a couple of minutes to run -- this is a computationally intensive method. 

In [ ]:
pyklip.parallelized.klip_dataset(dataset, outputdir="./", fileprefix="epoch1", annuli=11, 
                                 subsections=1, numbasis=[20], maxnumbasis=100, mode="ADI",
                                 movement=1)

# Show the output image

Do you see any planets?

In [ ]:
output_img = np.nanmean(dataset.output[0,:,0], axis=0) # combine images in time

plt.figure()
plt.imshow(output_img, cmap="inferno", vmin=np.nanpercentile(output_img, 1), vmax=np.nanpercentile(output_img, 99.7))
plt.xlim([dataset.output_centers[0,0]-200, dataset.output_centers[0,0]+200])
plt.ylim([dataset.output_centers[0,1]-200, dataset.output_centers[0,1]+200])
plt.title("Stellar PSF Subtracted Image")

# Measure the position and brighness of a planet 

The code below works for a single planet by fitting the planet to a 2-D Gaussian. This is not the most optimal method, but it is fast and good enough for demonstration purposes. 

**Note**: you need to change the `guess_x` and `guess_y` values for each planet you want to measure the properties these. These two values are an approximate (x,y) pixel location for the position of the planet in the processed image shown above. The guess needs to be fairly accurate (< 10 pixels), so you may need to zoom into portions of the image (or open the output FITS file in the FITS viewer like DS9) to come up with a good guess. 

The code cell below will measure the planet's flux, compute the relative offset from the star, and output the separation (in milliarcseconds) and position angle (in degrees) of the planet with respect to the star.

Note that the planet flux is still in arbitrary units. The cell after this one will calibrate it to the star.

In [ ]:
guess_x, guess_y = 575, 583
guess_flux = 800

peakflux, fwhm, planet_x, planet_y = pyklip.fakes.gaussfit2d(output_img, guess_x, guess_y, guesspeak=guess_flux)
print(peakflux, planet_x, planet_y)

star_y, star_x = dataset.output_centers[0]

planet_sep_pixels = np.sqrt((planet_x - star_x)**2 + (planet_y - star_y)**2)
planet_PA = np.degrees(np.arctan2(-(planet_x - star_x), planet_y - star_y)) % 360

platescale = 9.952 # mas/pixel
planet_sep_mas = planet_sep_pixels * platescale

print(planet_sep_mas, planet_PA)

## Measure the flux ratio of the planet

We will measure the flux of the star in the calibration frame, account for the difference in exposure times, and measure the flux ratio of the planet with respect to the star. 

In [ ]:
peakflux_star, _, _, _ = pyklip.fakes.gaussfit2d(calib_frame, calib_centx, calib_centy, guesspeak=10000)

flux_ratio = (peakflux/exptime)/(peakflux_star/calib_exptime)
print(flux_ratio)